# Simple GRU network with pretrained vectors for initialization

In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import tensorflow as tf
#from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Permute, GRU, Conv1D, LSTM, Embedding, Dropout, Activation, CuDNNLSTM, CuDNNGRU, concatenate, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalAveragePooling1D, BatchNormalization, SpatialDropout1D, Dot
from keras.optimizers import Adam
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras_tqdm import TQDMNotebookCallback
import keras.backend as K
from keras.callbacks import LearningRateScheduler
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from functools import reduce
from keras.layers import Layer, PReLU, SpatialDropout1D
from keras import initializers
from sklearn.model_selection import cross_val_predict

from nltk.tokenize import word_tokenize, wordpunct_tokenize, TweetTokenizer, MWETokenizer, ToktokTokenizer
from nltk.corpus import stopwords

import unicodedata
from collections import Counter
import itertools

np.random.seed(786)

/home/mohsin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [48]:
path = '../input/'
utility_path = '../utility/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
EMBEDDING_FILE=f'{utility_path}crawl-300d-2M.vec'
TRAIN_DATA_FILE=f'{path}train.csv'
TEST_DATA_FILE=f'{path}test.csv'

In [49]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [50]:
def unicodeToAscii(series):
    return series.apply(lambda s: unicodedata.normalize('NFKC', str(s)))


def multiple_replace(text, adict):
    rx = re.compile('|'.join(map(re.escape, adict)))

    def one_xlat(match):
        return adict[match.group(0)]

    return rx.sub(one_xlat, text)

STOP_WORDS = set(stopwords.words( 'english' ))
# Lowercase, trim, and remove non-letter characters
def normalizeString(series):
    series = unicodeToAscii(series)
    series = series.str.lower()
    series = series.str.replace(r"(\n){1,}", "")
    series = series.str.replace(r"\'", "")
    series = series.str.replace(r"\-", "")
    series = series.str.replace(r"[^0-9a-zA-Z]+", " ")

    return series


def token_generator(texts):
    for text in texts:
        tokens = str(text).split()
        tokens = [token for token in tokens if token not in STOP_WORDS]
        yield tokens


class Tokenizer:
    def __init__(self, max_features=20000, max_len=10, tokenizer=str.split):
        self.max_features = max_features
        self.tokenizer = tokenizer
        self.doc_freq = None
        self.vocab = None
        self.vocab_idx = None
        self.max_len = max_len

    def fit_transform(self, texts):
        tokenized = []
        n = len(texts)

        tokenized = token_generator(texts)
        doc_freq = Counter(itertools.chain.from_iterable(tokenized))

        vocab = [t[0] for t in doc_freq.most_common(self.max_features)]
        vocab_idx = {w: (i + 1) for (i, w) in enumerate(vocab)}
        # doc_freq = [doc_freq[t] for t in vocab]

        # self.doc_freq = doc_freq
        self.vocab = vocab
        self.vocab_idx = vocab_idx

        result_list = []
        tokenized = token_generator(texts)
        for text in tokenized:
            text = self.text_to_idx(text, self.max_len)
            result_list.append(text)

        result = np.zeros(shape=(n, self.max_len), dtype=np.int32)
        for i in range(n):
            text = result_list[i]
            result[i, :len(text)] = text

        return result

    def text_to_idx(self, tokenized, max_len):
        return [self.vocab_idx[t] for i, t in enumerate(tokenized) if (t in self.vocab_idx) and (i < max_len)]

    def transform(self, texts):
        n = len(texts)
        result = np.zeros(shape=(n, self.max_len), dtype=np.int32)

        for i in range(n):
            text = self.tokenizer(texts[i])
            text = self.text_to_idx(text, self.max_len)
            result[i, :len(text)] = text

        return result

    def vocabulary_size(self):
        return len(self.vocab) + 1


def grams2(src_words):
    return list(zip(src_words, src_words[1:]))


def tokenizer_2gram(text):
    return grams2(str(text).split())


def ngrams(n, f, prune_after=10000):
    counter = collections.Counter()
    deque = collections.deque(maxlen=n)
    for i, line in enumerate(f):
        deque.clear()
        words = str(line).split()
        deque.extend(words[:n - 1])
        for word in words[n - 1:]:
            # if word in vocab:
            deque.append(word)
            ngram = tuple(str(w) for w in deque)
            if i < prune_after or ngram in counter:
                counter[ngram] += 1
    return counter


In [51]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

print(train.shape, test.shape)

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values

#Get validation folds
train['target_str'] = reduce(lambda x,y: x+y, [train[col].astype(str) for col in list_classes])
train['target_str'] = train['target_str'].replace('110101', '000000').replace('110110','000000')
cvlist1 = list(StratifiedKFold(n_splits=10, random_state=786).split(train, train['target_str'].astype('category')))
cvlist2 = list(StratifiedShuffleSplit(n_splits=5, test_size=0.05, random_state=786).split(train, train['target_str'].astype('category')))

(159571, 8) (153164, 2)


/home/mohsin/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [52]:
for df in train, test:
    df["comment_text"] = normalizeString(df["comment_text"])

In [53]:
MAX_FEATURES = 200000
MAX_LEN = 250

tok = Tokenizer(max_features=MAX_FEATURES, max_len=MAX_LEN)
X_train = tok.fit_transform(train["comment_text"].astype(str))
X_test = tok.transform(test["comment_text"].astype(str))

print(X_train.shape, X_test.shape)

(159571, 250) (153164, 250)


In [54]:
del train, test

In [55]:
EMBED_SIZE = 300
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')

def initialize_embeddings(filename, tokenizer):
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(filename))

    word_index = tokenizer.vocab_idx
    nb_words = min(MAX_FEATURES, len(word_index))
    embedding_matrix = np.zeros((nb_words, EMBED_SIZE))
    for word, i in word_index.items():
        if i >= MAX_FEATURES: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [56]:
embedding_matrix = initialize_embeddings(EMBEDDING_FILE, tok)
print(embedding_matrix.shape)
print(np.mean(embedding_matrix), np.std(embedding_matrix))

(200000, 300)
0.002500578637546975 0.2183801808372828


In [57]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        # self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))),
                        (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        # print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        # return input_shape[0], input_shape[-1]
        return input_shape[0], self.features_dim

In [58]:
class ZeroMaskedEntries(Layer):
    """
    This layer is called after an Embedding layer.
    It zeros out all of the masked-out embeddings.
    It also swallows the mask without passing it on.
    You can change this to default pass-on behavior as follows:

    def compute_mask(self, x, mask=None):
        if not self.mask_zero:
            return None
        else:
            return K.not_equal(x, 0)
    """

    def __init__(self, **kwargs):
        self.support_mask = True
        super(ZeroMaskedEntries, self).__init__(**kwargs)

    def build(self, input_shape):
        self.output_dim = input_shape[1]
        self.repeat_dim = input_shape[2]

    def call(self, x, mask=None):
        #print(mask.shape)
        mask = K.cast(mask, 'float32')
        mask = K.repeat(mask, self.repeat_dim)
        #print(mask.shape)
        mask = K.permute_dimensions(mask, (0, 2, 1))
        return x * mask

    def compute_mask(self, input_shape, input_mask=None):
        return None
    
def mask_aware_mean(x):
    # recreate the masks - all zero rows have been masked
    #mask = K.not_equal(K.sum(K.abs(x), axis=2, keepdims=True), 0)

    # number of that rows are not all zeros
    #n = K.sum(K.cast(mask, 'float32'), axis=1, keepdims=False)
    # compute mask-aware mean of x
    x_mean = K.sum(x, axis=1, keepdims=False)
    #print(x_mean.shape)
    return x_mean

def mask_aware_mean_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 3 
    return (shape[0], shape[2])

In [68]:
from sklearn.base import BaseEstimator, ClassifierMixin
class GRUClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, gru_dim=150, dense_dim=256, batch_size=128, epochs=2, bidirectional=False, 
                 pool_type='all', initial_weights=None, optimizer='adam' ,verbose=1, out_dim=6, callbacks=None):
        
        self.gru_dim = gru_dim
        self.dense_dim = dense_dim
        self.batch_size = batch_size
        self.epochs= epochs
        self.bidirectional = bidirectional
        self.pool_type = pool_type
        self.initial_weights = initial_weights
        self.verbose = verbose
        self.callbacks = callbacks
        self.optimizer = optimizer
        self.out_dim = out_dim
    def _build_model(self):
        inp = Input(shape=(MAX_LEN,))
        emb = Embedding(MAX_FEATURES, 
                        EMBED_SIZE,
                        weights=[self.initial_weights],
                        mask_zero=True,
                        #embeddings_regularizer=regularizers.l2(0.0001),
                        trainable=True)(inp)

        emb = ZeroMaskedEntries()(emb)
        if self.bidirectional:
            enc = Bidirectional(CuDNNGRU(int(self.gru_dim), return_sequences=True, return_state=True, 
                                         ))(emb)
            x = enc[0]
            state = enc[1]
        else:
            x, state = CuDNNGRU(int(self.gru_dim), return_sequences=True, return_state=True,
                            #kernel_regularizer=regularizers.l2(0.0001),
                            # recurrent_regularizer=regularizers.l2(0.0001),
                             #bias_regularizer=regularizers.l2(0.0001)
                               )(emb)
            #x = SpatialDropout1D(0.5)(x)
        
        if self.pool_type == 'avg':
            x = GlobalAveragePooling1D()(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'max':
            x = GlobalMaxPool1D()(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'attn':
            x = Attention(MAX_LEN)(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'all':
            #x1 = GlobalAveragePooling1D()(x)
            x2 = GlobalMaxPool1D()(x)
            x3 = Attention(MAX_LEN)(x)
            x = concatenate([x2, x3, state])
    
        #x = Dropout(0.5)(x)
        x = Dense(self.dense_dim)(x)
        x = PReLU()(x)

        out = Dense(self.out_dim, activation="sigmoid")(x)
        
        model = Model(inputs=inp, outputs=out)
        model.compile(loss='binary_crossentropy', optimizer=self.optimizer, metrics=['accuracy'])
        return model
    
    def fit(self, X, y):
        self.model = self._build_model()
        
        if self.callbacks:
            self.model.fit(X, y, batch_size=self.batch_size, epochs=self.epochs,
                       verbose=self.verbose,
                       callbacks=self.callbacks,
                       shuffle=True)
        else:
            self.model.fit(X, y, batch_size=self.batch_size, epochs=self.epochs,
                       verbose=self.verbose,
                       shuffle=True)
        return self
    
    def predict(self, X, y=None):
        if self.model:
            y_hat = self.model.predict(X, batch_size=2048)
        else:
            raise ValueError("Model not fit yet")
        return y_hat

In [69]:
def lr_decay(epoch):
    if epoch == 0:
        return 0.0015
    if epoch == 1:
        return 0.0007
    if epoch == 2:
        return 0.001
    if epoch == 3:
        return 0.00001


def shuffle_crossvalidator(model, cvlist, X, y):
    y_trues = []
    y_preds = []
    LRDecay = LearningRateScheduler(lr_decay)

    for tr_index, val_index in cvlist2:
        X_tr, y_tr = X[tr_index, :], y[tr_index, :]
        X_val, y_val = X[val_index, :], y[val_index, :]
        RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

        model.set_params(**{'callbacks':[RocAuc, LRDecay]})
        model.fit(X_tr, y_tr)

        y_pred = model.predict(X_val)
        print("ROC AUC for this fold is ", roc_auc_score(y_val, y_pred))
        y_trues.append(y_val)
        y_preds.append(y_pred)
        K.clear_session()
        #break
    score = roc_auc_score(np.concatenate(y_trues), np.concatenate(y_preds))
    print("Overall score on 10 fold CV is {}".format(score))
    
    return y_preds, y_trues, score

def outoffold_crossvalidator(model_params, cvlist, X, y):
    y_preds = np.zeros(y.shape)
    LRDecay = LearningRateScheduler(lr_decay)

    for tr_index, val_index in cvlist2:
        X_tr, y_tr = X[tr_index, :], y[tr_index, :]
        X_val, y_val = X[val_index, :], y[val_index, :]
        RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)
        
        model.set_params(**{'callbacks':[RocAuc, LRDecay]})
        model.fit(X_tr, y_tr)

        y_pred = model.predict(X_val)
        print("ROC AUC for this fold is ", roc_auc_score(y_val, y_pred))
        y_preds[val_idx] = y_pred
        K.clear_session()
        #break
    score = roc_auc_score(y, y_preds)
    print("Overall score on 10 fold CV is {}".format(score))
    
    return y_preds, y_trues, score


In [ ]:
model = GRUClassifier(gru_dim=500, dense_dim=600, initial_weights=embedding_matrix, bidirectional=False,
                    batch_size=128, epochs=2, optimizer='adam')

y_preds, y_trues, _ = shuffle_crossvalidator(model, cvlist2, X_train, y)

Epoch 1/2
151552/151592 [============================>.] - ETA: 0s - loss: 0.0500 - acc: 0.9812
 ROC-AUC - epoch: 1 - score: 0.988671 

151592/151592 [==============================] - 100s 660us/step - loss: 0.0500 - acc: 0.9812
Epoch 2/2
142720/151592 [===========================>..] - ETA: 5s - loss: 0.0324 - acc: 0.9867

In [ ]:

#train_gru()
model = GRUClassifier(initial_weights=embedding_matrix, bidirectonal=True,
                    batch_size=128, epochs=2, optimizer='adam')

y_preds, y_trues, _ = shuffle_crossvalidator(model, cvlist2, X_train, y)

In [23]:
import gc 
gc.collect()
K.clear_session()

In [27]:
np.concatenate(y_preds).shape
y_trues.shape

NameError: name 'y_trues' is not defined

In [15]:
import skopt
from skopt import gp_minimize, gbrt_minimize
from skopt.space import Real, Integer, Categorical

In [20]:
space = [Integer(16, 256), #name='embed_size'),
         Integer(16, 256),# name='gru_dim'),
         Integer(64, 1024),# name='dense_dim'),
         Real(0.0001, 0.005, "log-uniform"), #name='lr1'),
         Real(1e-5, 0.002, "log-uniform"), #name='lr2'),
         Real(1e-8, 0.001, "log-uniform"), #name='decay'),
         Integer(32, 256),# name=batch'),
        ]

res_gp = gbrt_minimize(train_gru, space, n_calls=100, random_state=0)

(?, 200, 188)
Epoch 1/2
151389/151592 [============================>.] - ETA: 0s - loss: 0.0622 - acc: 0.9795
 ROC-AUC - epoch: 1 - score: 0.982213 

151592/151592 [==============================] - 36s 237us/step - loss: 0.0621 - acc: 0.9795
Epoch 2/2
151389/151592 [============================>.] - ETA: 0s - loss: 0.0363 - acc: 0.9857
 ROC-AUC - epoch: 2 - score: 0.983151 

151592/151592 [==============================] - 36s 238us/step - loss: 0.0364 - acc: 0.9857
ROC AUC for this fold is  0.9831507887571517
(?, 200, 188)
Epoch 1/2
151389/151592 [============================>.] - ETA: 0s - loss: 0.0624 - acc: 0.9793
 ROC-AUC - epoch: 1 - score: 0.982551 

151592/151592 [==============================] - 36s 234us/step - loss: 0.0624 - acc: 0.9793
Epoch 2/2
151389/151592 [============================>.] - ETA: 0s - loss: 0.0368 - acc: 0.9855
 ROC-AUC - epoch: 2 - score: 0.984904 

151592/151592 [==============================] - 36s 234us/step - loss: 0.0368 - acc: 0.9855
ROC AUC for

KeyboardInterrupt: 

In [129]:
y_trues

NameError: name 'y_trues' is not defined

In [43]:
roc_auc_score(y, y_preds)

0.9861737784746518

In [41]:
model = get_model()
model.fit(X_t, y, batch_size=32, epochs=3, validation_split=0.0, verbose=1, 
              callbacks=[RocAuc, LRDecay])
y_test_preds = model.predict([X_te], batch_size=1024, verbose=1)


Epoch 1/3
159456/159571 [============================>.] - ETA: 0s - loss: 0.0498 - acc: 0.9818
 ROC-AUC - epoch: 1 - score: 0.992135 

159571/159571 [==============================] - 84s 528us/step - loss: 0.0498 - acc: 0.9818
Epoch 2/3
159488/159571 [============================>.] - ETA: 0s - loss: 0.0384 - acc: 0.9850
 ROC-AUC - epoch: 2 - score: 0.995249 

159571/159571 [==============================] - 83s 523us/step - loss: 0.0384 - acc: 0.9850
Epoch 3/3
159488/159571 [============================>.] - ETA: 0s - loss: 0.0307 - acc: 0.9880
 ROC-AUC - epoch: 3 - score: 0.995429 

153164/153164 [==============================] - 2s 15us/step


In [25]:
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
y_trues = train[label_cols].values
y_preds2 = np.zeros((X_t.shape[0], len(label_cols)))
y_test_preds2 = np.zeros((X_te.shape[0], len(label_cols)))
for i, col in enumerate(label_cols):
    y = y_trues[:, i]
    #model = RandomForestClassifier(n_estimators=100, max_depth=6, min_samples_leaf=50, class_weight='balanced', n_jobs=-1)
    model = lgb.LGBMClassifier(n_estimators=100, num_leaves=5, learning_rate=0.03, 
                               subsample=0.9, colsample_bytree=0.9)
    y_preds2[:, i] = cross_val_predict(model, y_preds, y, cv=cvlist, n_jobs=1, method='predict_proba')[:,1]
    y_test_preds2[:, i] = model.fit(y_preds, y).predict_proba(y_test_preds)[:,1]
    print("Score for class {} is {}".format(col, roc_auc_score(y, y_preds2[:, i])))
print("Over auc score", roc_auc_score(y_trues, y_preds2))

Score for class toxic is 0.9791088111561013
Score for class severe_toxic is 0.9896361881404786
Score for class obscene is 0.989792727484462
Score for class threat is 0.9834375512104746
Score for class insult is 0.9851961777705942
Score for class identity_hate is 0.9837426177272286
Over auc score 0.9851523455815565


In [42]:
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[label_cols] = y_test_preds
sample_submission.to_csv('nn_submission.csv', index=False)

In [ ]:
sample_submission[label_cols] = y_test_preds2
sample_submission.to_csv('nn_lgbmeta_submission.csv', index=False)